In [3]:

import librosa
import soundfile
import os, glob
import numpy as np


In [4]:
# Extract features : mfcc, chroma, mel

def extract_feature(file, mfcc, chroma, mel):
    with soundfile.SoundFile(file) as file:
        X = file.read(dtype="float32")
        sample_rate=file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        
        return result

In [5]:
# Emotions in the RAVDESS dataset

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# Emotions to observe
observed_emotions = ['neutral','happy','angry','surprised']

In [6]:

# Load the data and extract features for each sound file

# wav_files = 'C:\\Users\\admin\\Downloads\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav'
wav_files = 'C:\\Users\\admin\\Downloads\\bdad-project\\sample-ravdess\\Actor_*\\*.wav'

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(wav_files):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Split the dataset
x_train,x_test,y_train,y_test = load_data(test_size=0.25)


In [18]:
model= MLPClassifier(
            alpha=0.01, 
            batch_size=10, 
            epsilon=1e-08, 
            hidden_layer_sizes=(300,), 
            learning_rate='adaptive', 
            max_iter=500 )

model.fit(x_train, y_train)


MLPClassifier(alpha=0.01, batch_size=10, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [21]:
model.score(x_test, y_test)

0.6285714285714286

In [14]:
# recording sound and storing as a wave file

import sounddevice 
from scipy.io.wavfile import write

fs = 44100
seconds = 5
print("recording...")
record_voice = sounddevice.rec(int(seconds*fs), samplerate=fs, channels=2)
sounddevice.wait()
write("C:\\Users\\admin\\Downloads\\bdad-project\\rec-voice.wav", fs, record_voice)
print("DONE")


recording...
DONE


In [16]:
from pydub import AudioSegment

sound = AudioSegment.from_mp3("C:/Users/admin/Downloads/record.mp3")
sound.export("C:/Users/admin/Downloads/outputwave.wav", format="wav")


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [12]:
import os
import glob

# files                                                                         
lst = glob.glob("C:\\Users\\admin\\Downloads\\*.mp3")
print(lst)
 

['C:\\Users\\admin\\Downloads\\record.mp3']


In [17]:
for file in lst:
    # convert wav to mp3
    os.system(f"""ffmpeg -i {file} -acodec pcm_u8 -ar 22050 {file[:-4]}.wav""")   